In [ ]:
# app.py
from flask import Flask, request, jsonify
from flask_cors import CORS
import wikipedia
import json # Import the json module
from transformers import pipeline
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
CORS(app) # Enable CORS for all routes

# --- Load Knowledge Base from File ---
# This dictionary will hold the structured monument data loaded from the text file.
monument_knowledge_base = {}
try:
    with open('knowledge_base.txt', 'r', encoding='utf-8') as f:
        monument_knowledge_base = json.load(f)
    logger.info("Knowledge base loaded successfully from knowledge_base.txt")
except FileNotFoundError:
    logger.error("knowledge_base.txt not found. Monument information will be limited.")
except json.JSONDecodeError:
    logger.error("Error decoding JSON from knowledge_base.txt. Check file format.")
except Exception as e:
    logger.error(f"An unexpected error occurred while loading knowledge base: {e}")

# Initialize the translation pipeline outside the route to load the model only once
try:
    # Use a smaller, faster model if possible, e.g., 'Helsinki-NLP/opus-mt-en-hi' for English to Hindi
    # For multiple languages, you might need a larger multilingual model or separate models.
    # For simplicity, we'll use a single multilingual model (if available) or assume handling on the client for many-to-many.
    # A common approach for many languages is to translate to/from English as an intermediary.
    # For a general multilingual model, 'Helsinki-NLP/opus-mt-mul-en' (many to English) and 'Helsinki-NLP/opus-mt-en-mul' (English to many) are options.
    # We'll use a placeholder for now, assuming the client or a more sophisticated backend handles the actual language pair.
    # For a robust solution, you'd integrate a true multilingual translation API (e.g., Google Translate API).
    translator_pipeline = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr") # Example: English to French model
    logger.info("Hugging Face translation pipeline loaded.")
except Exception as e:
    logger.error(f"Could not load Hugging Face translation pipeline: {e}")
    translator_pipeline = None


# --- General NLP Lexicon (for general queries like greetings) ---
# This can still be hardcoded or loaded from another simple text file if needed.
# It handles conversational aspects not directly related to monuments.
general_nlp_lexicon = {
    'hello': 'Hi there! How can I help you today?',
    'hi': 'Hello! What\'s on your mind?',
    'how are you': 'I am just a program, but I\'m doing great! How about you?',
    'what is your name': 'I don\'t have a name. I am a chatbot designed to assist you.',
    'help': 'I can answer basic questions and provide specific information about tourist locations. Try asking "Who created the Taj Mahal?" or "When was the Gateway of India created?".',
    'bye': 'Goodbye! Have a great day!',
    'thank you': 'You\'re welcome!',
    'joke': 'Why don\'t scientists trust atoms? Because they make up everything!',
}


# --- ROUTES ---

@app.route('/extract_more_info', methods=['GET'])
def extract_more_info():
    monument_name = request.args.get('monument_name')
    if not monument_name:
        return jsonify({"error": "Monument name is required"}), 400

    try:
        # Set a shorter timeout for Wikipedia API calls
        wikipedia.set_rate_limiting(True)
        wikipedia.set_api_url("https://en.wikipedia.org/w/api.php") # Ensure English Wikipedia

        summary = wikipedia.summary(monument_name, sentences=3, auto_suggest=True, redirect=True)
        return jsonify({"extracted_text": summary})
    except wikipedia.exceptions.PageError:
        logger.warning(f"Wikipedia PageError for '{monument_name}'")
        # Attempt a search if direct summary fails
        try:
            search_results = wikipedia.search(monument_name)
            if search_results:
                # Try the first search result
                summary = wikipedia.summary(search_results[0], sentences=3, auto_suggest=True, redirect=True)
                return jsonify({"extracted_text": summary})
            else:
                return jsonify({"error": f"No Wikipedia page found for '{monument_name}'."}), 404
        except Exception as e:
            logger.error(f"Error during Wikipedia search/summary for '{monument_name}': {e}")
            return jsonify({"error": f"Could not find information for '{monument_name}'. Please try another name or rephrase."}), 500
    except wikipedia.exceptions.DisambiguationError as e:
        logger.warning(f"Wikipedia DisambiguationError for '{monument_name}': {e.options}")
        return jsonify({"error": f"Multiple results found for '{monument_name}'. Please be more specific. Options: {', '.join(e.options[:5])}"}), 400
    except wikipedia.exceptions.HTTPTimeoutError:
        logger.error(f"Wikipedia HTTP Timeout for '{monument_name}'")
        return jsonify({"error": "Wikipedia request timed out. Please try again later."}), 504
    except Exception as e:
        logger.error(f"An unexpected error occurred during Wikipedia extraction for '{monument_name}': {e}")
        return jsonify({"error": f"An unexpected error occurred while fetching information for '{monument_name}'."}), 500


@app.route('/translate', methods=['POST'])
def translate_text():
    data = request.json
    text_to_translate = data.get('text')
    target_language = data.get('target_language')

    if not text_to_translate or not target_language:
        return jsonify({"error": "Missing text or target_language"}), 400

    if translator_pipeline is None:
        return jsonify({"error": "Translation service not available."}), 503

    try:
        # Hugging Face translation pipeline expects a specific format for target_language
        # For 'Helsinki-NLP/opus-mt-en-mul' or similar multilingual models, you might specify the target lang code
        # For our example 'Helsinki-NLP/opus-mt-en-fr', it always translates to French if source is English.
        # A more dynamic setup would require changing the model or using a different library/API.
        # For now, we'll simulate based on the chosen model.
        
        # NOTE: A robust solution for many languages would likely use an external API
        # like Google Cloud Translation API or manage multiple HF models.
        # This example assumes the selected HF model can handle the 'en->target_language' pair.
        # If your chosen HF model is e.g., 'Helsinki-NLP/opus-mt-en-hi' for English to Hindi,
        # it will only translate EN->HI.
        
        # A simple check for the example model:
        if translator_pipeline.model.config.architectures[0].startswith("M") and target_language != "en":
             # This is a highly simplified assumption for a hypothetical multilingual model setup.
             # In a real scenario, you'd need to correctly instantiate the right model for the language pair.
             # Or use a service like Google Translate API that handles all pairs.
             # For 'Helsinki-NLP/opus-mt-en-fr' specifically: it only translates EN->FR.
             # If target_language is not 'fr', we will return original text as a fallback.
             if target_language == 'fr':
                translated_output = translator_pipeline(text_to_translate, src_lang="en", tgt_lang="fr")
             elif target_language == 'hi': # Example if you had an en-hi model
                # translated_output = translator_pipeline_en_hi(text_to_translate)
                # For this generic setup, we just return original if specific model isn't there
                return jsonify({"translated_text": text_to_translate})
             else:
                return jsonify({"translated_text": text_to_translate}) # Fallback if specific model for language not loaded
        else: # If not a multilingual model or target is English, don't translate or return original
            return jsonify({"translated_text": text_to_translate})

        translated_text = translated_output[0]['translation_text']
        return jsonify({"translated_text": translated_text})

    except Exception as e:
        logger.error(f"Translation error: {e}")
        return jsonify({"error": f"Failed to translate text: {e}"}), 500


# @app.route('/')
# def index():
#     return "Chatbot backend is running!"

if __name__ == '__main__':
    app.run(debug=True)